# 2. Dam water level monitoring using ALOS-2/PALSAR-2 observations

- This section is designed for learners who are interested in monitor water level of a dam with ALOS-2 - case study of Yamba Dam, Japan.

## Learning objectives of this training material

- Pre-processing to reduce speckle noise (optional)
- Visualization of SAR satellite observation data of Yamba Dam
- Change detection of dam operation


### Introduction to ALOS-2

#### a. (Basic) [Introduction to ALOS-2 data imagery](https://www.eoportal.org/satellite-missions/alos-2#alos-2-advanced-land-observing-satellite-2-sar-mission--daichi-2)

This website provides general information about the ALOS-2 (Daichi-2) satellite, their mission and its advanced SAR capabilities for Earth observation and disaster monitoring.
Learners wil understand the points below from this content:
1. Overview of ALOS-2.
2. Mission Capabilities
3. Performance Specifications
4. Space and Hardware Components
5. ALOS-2 (Advanced Land Observing Satellite-2; SAR mission) / Daichi-2
6. Spacecraft
7. Mission Status
8. Sensor Complement 
9. Ground System

#### b. (Basic) [ALOS Data Application (Examples of Researches)](https://www.eorc.jaxa.jp/ALOS/en/gallery/example.htm)

NASDA's EORC classifies ALOS data application research areas into the following 12 categories;

- PRISM / AVNIR-2: Evaluation of PRISM and AVNIR-2 involves assessing sensor characteristics, image quality, geometric and radiometric calibration and validation, and developing methods for fast and accurate extraction of geo-physical parameters.
- PALSAR: Evaluation of PALSAR focuses on assessing sensor characteristics, image quality, geometric and radiometric calibration and validation, along with enhancing analysis methods using microwave scattering and SAR interference.
- Digital Elevation Model (DEM) and Mapping
- Land Use Monitoring and Land Cover Classification
- Disaster Monitoring
- Geological Surveying and Mineral Resource Exploration
- Terrestrial Ecosystem and Forestry Management
- Oceanography and Coastal Zone Related Research
- Snow and Ice Related Research
- Agriculture Management
- Hydrology and Water Resource Management
- Application Research using Geographical Information

First, install the packages used in this exercise.

In [ ]:
!conda install -c conda-forge numpy matplotlib scikit-image rasterio -y
!pip install opencv-python

## 2.1. Preparing ALOS-2 data

### 2.1.1. Downloading ALOS-2 data

In [ ]:
!curl --output 2-Yamba.zip "https://owncloud.glodal-inc.net/owncloud/index.php/s/fORunC9aDo38G5s/download"

import shutil, os
shutil.unpack_archive("2-Yamba.zip", ".")
os.chdir('Yamba')
os.listdir()

### 2.1.2 Preprocessing (speckle noise reduction and DN transform)

#### Speckle noise reduction using Lee filter

The Lee filter is a technique used to reduce speckle noise in SAR (Synthetic Aperture Radar) images. Speckle noise is the granular, random noise that occurs naturally in SAR observations and often affects image analysis.

The following code implements a **Lee filter** commonly used to reduce speckle noise in synthetic aperture radar (SAR) images. This filter works by processing each pixel in the image using a sliding window (of the size defined by `window_size`). For each window, it computes the local mean and variance and determines a damping factor that controls how much smoothing is applied to the pixel.

References:
- [SAR Image Despeckling Using Refined Lee Filter](https://ieeexplore.ieee.org/document/7334965)
- [Digital Image Enhancement and Noise Filtering by Use of Local Statistics](https://ieeexplore.ieee.org/document/4766994?rsource=https:%2F%2Flinks.esri.com%2FNoiseFilteringUsingLocalStats)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from skimage import img_as_float
from scipy.ndimage import uniform_filter


import numpy as np
from scipy.ndimage import uniform_filter

# Define a lee filter function


def lee_filter(image, size):
    """
    Applies the Lee filter to an input image to reduce speckle noise.
    
    Parameters:
        image (numpy.ndarray): The input image to be filtered.
        size (int or tuple): The size of the local neighborhood (e.g., window size).
        
    Returns:
        numpy.ndarray: The filtered image.
    """
    # Ensure the input image is a floating-point array for precision
    image = image.astype(np.float32)
    
    # Calculate local mean and squared mean using uniform filter
    local_mean = uniform_filter(image, size=size)
    local_mean_sqr = uniform_filter(image**2, size=size)
    
    # Calculate local variance
    local_variance = local_mean_sqr - local_mean**2
    
    # Estimate noise variance (overall variance of the image)
    overall_variance = np.mean(local_variance)
    
    # Compute the filter weight
    weight = local_variance / (local_variance + overall_variance)
    weight = np.clip(weight, 0, 1)  # Ensure weight is in a valid range
    
    # Apply the Lee filter formula
    filtered_image = local_mean + weight * (image - local_mean)
    
    return filtered_image


#### Conversion from DN values to backscatter coefficients

In time series analysis using multiple observation data, it is necessary to convert pixel values to backscatter coefficients. For details, see [Calibration and Validation](https://www.eorc.jaxa.jp/ALOS/jp/alos-2/a2_calval_j.htm)

$$ \sigma^0_{Q16} = 10 log_{10}<DN^2> + CF$$

Note that <> represents averaging for noise reduction and -83.0 dB is used for CF. A coding example is shown below.

In [ ]:
def calibrate_image(image):
    image = img_as_float(image)
    calibrated_image = (10 * np.log10(image**2)) - 83
    return calibrated_image

##### Performs preprocessing on each image data

Processes the functions defined in the previous section on each image data and displays the results.

In [ ]:
import os, math, re

# File handling
image_files = [f for f in os.listdir() if f.endswith('.tif')]
preprocessed_images = []

for file in image_files:
    with rasterio.open(file) as src:
        if src.count >= 1:
            img = src.read(1)  # Read the first band
        else:
            raise ValueError(f"File {file} does not contain a valid band.")
    
    # Step 1: Apply speckle filter
    filtered_img = lee_filter(img, size=5)
    
    # Step 2: Convert to backscatter coefficients
    calibrated_img = calibrate_image(filtered_img)
    preprocessed_images.append(calibrated_img)

# Define number of rows and compute columns dynamically
nrows = 3
ncols = math.ceil(len(preprocessed_images) / nrows)

# Create subplots grid
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 4, nrows * 4))

# Flatten axes to simplify indexing (even if nrows > 1)
axes = axes.flatten()

# Plot each image
for i, image in enumerate(preprocessed_images):
    axes[i].imshow(image, cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(re.search(r"-(\d{6})-", image_files[i]).group(1))

# Turn off unused axes if there are any
for ax in axes[len(preprocessed_images):]:
    ax.axis('off')

plt.tight_layout()
plt.show()


The area with the smallest backscatter coefficient (dark area around the center) represents the water area. The water area appears from October 11 to November 3, which is [waterlogged due to heavy rainfall caused by Typhoon No. 19 in 2019](https://ja.wikipedia.org/wiki/%E5%85%AB%E3%83%83%E5%A0%B4%E3%83%80%E3%83%A0#%E4%BB%A4%E5%92%8C%E5%85%83%E5%B9%B4%E6%9D%B1%E6%97%A5%E6%9C%AC%E5%8F%B0%E9%A2%A8). The fact that the surface of the water was visible for several months afterward suggests that the water was released over time.

## 2.2. Analyze changes by color composite image

Changes can be analyzed by color compositing SAR satellite data with different observation dates. Here is an example of assigning October 11, 2020 (pre-typhoon) to the red channel and November 3, 2019 (post-typhoon) to the green and blue channels.

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

red   = preprocessed_images[image_files.index('IMG-HH-ALOS2290760720-191011-UBSR2.1GUA.tif')]
green = preprocessed_images[image_files.index('IMG-HH-ALOS2294160720-191103-UBSR2.1GUA.tif')]
blue  = preprocessed_images[image_files.index('IMG-HH-ALOS2294160720-191103-UBSR2.1GUA.tif')]

# Stack bands into RGB image array
rgb_image = np.stack([red, green, blue], axis=-1)

# Normalize bands for better visibility (optional)
rgb_image = (rgb_image - rgb_image.min()) / (rgb_image.max() - rgb_image.min())  # Normalize to [0, 1]

# Show RGB composite
plt.figure(figsize=(10, 10))
plt.imshow(rgb_image)
plt.title("Color Composite with 191011 and 191103")
plt.axis("off")
plt.show()

Red areas indicate areas that were flooded by the heavy rain on October 12. Black areas indicate areas that were still under water on October 11.

#### (Optional) Save the RGB composite image as GeoTiff
If you want to display the composite image in a GIS or other application, save it as a GeoTIFF with the same spatial information (coordinate reference system, geotransform, etc.) as the input bands. The following code saves the RGB composite while preserving the CRS and transformations of one of the input bands (e.g., the red band).

In [ ]:
meta_tif = 'IMG-HH-ALOS2290760720-191011-UBSR2.1GUA.tif'
outfile = "export.tif"

# To get metadata, open the red band again
with rasterio.open(meta_tif) as src:
    # Update metadata to support 3 bands (RGB) and float32 data type.
    rgb_meta = src.meta.copy()
    rgb_meta.update({
        "count": 3,           # RGB 3-band
        "dtype": "float32",   # Normalized image data type
        "driver": "GTiff",    # Make sure the output format is GeoTIFF.
        "compress": "Deflate" # Compression option recommended due to file size bloat
    })

    # Write RGB images as GeoTIFF
    with rasterio.open(outfile, "w", **rgb_meta) as dst:
        # Write each channel into a separate band
        dst.write((rgb_image[:, :, 0] * 255).astype("uint8"), 1)  # Red channel
        dst.write((rgb_image[:, :, 1] * 255).astype("uint8"), 2)  # Green Channel
        dst.write((rgb_image[:, :, 2] * 255).astype("uint8"), 3)  # Blue Channel

print(f"RGB composite image saved to {outfile}")

## 2.3. Detecting changes in the water area of a dam

Significant changes can be detected by calculating the difference images between the two time periods. In this section, data analysis is performed using October 11 and November 3 as examples.
- Typhoon No. 19 hit Japan on October 12, 2019, so images from October 11, 2019 will be used for the pre-typhoon images.
- For the post-typhoon image, we will use the image from November 3, 2019. This is the first available image after the typhoon.

First, the observed data are visualized in the image and compared by time period.

In [ ]:
import rasterio
import matplotlib.pyplot as plt

pre_typhoon   = preprocessed_images[image_files.index('IMG-HH-ALOS2290760720-191011-UBSR2.1GUA.tif')]
post_typhoon  = preprocessed_images[image_files.index('IMG-HH-ALOS2294160720-191103-UBSR2.1GUA.tif')]

# Set up the figure for visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot each image and store one of them for the color bar
im = axes[0].imshow(pre_typhoon, cmap='gray')  # Assign to `im` for color bar reference
axes[0].set_title("Pre-Typhoon (No Water): 11 Oct 2019")
axes[0].axis("off")

axes[1].imshow(post_typhoon, cmap='gray')
axes[1].set_title("Post-Typhoon (Almost Full Water Level): 3 Nov 2019")
axes[1].axis("off")

# Add a color bar below all images without overlapping
cbar_ax = fig.add_axes([0.2, 0.01, 0.6, 0.02])  # Centered below all images
fig.colorbar(im, cax=cbar_ax, orientation='horizontal').set_label('Backscaetter coefficient')

plt.tight_layout()
plt.show()

- Pre-typhoon: on October 11, 2019, the dam was not fully opened (completed in October 2019 and officially opened on April 1, 2020). Therefore, only a small river in the dam area can be seen in the image.
- Post-Typhoon: On November 3, 2019, the water level of the dam was very high, even though the dam was not officially opened. This was due to Typhoon Hagibis hitting Japan on October 12, 2019, which brought heavy rains and caused the water level of the Yamba Dam to rise sharply.

### 2.3.1. Calculate the difference between the two time periods

Calculate the difference image to detect changes from October 12 to November 3 in the following cells.

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

diff_img_1012_1103 = post_typhoon - pre_typhoon

Display the difference image. Areas flooded by the typhoon are highlighted.

In [ ]:
import matplotlib.pyplot as plt

# Visualize results with a legend
fig, axes = plt.subplots(1, 3, figsize=(24, 6))

# Plot the pre-typhoon image.
img0 = axes[0].imshow(pre_typhoon, cmap='gray')
axes[0].set_title("Pre-Typhoon Image")
axes[0].axis("off")
cbar0 = fig.colorbar(img0, ax=axes[0], orientation='vertical', fraction=0.046, pad=0.04)
cbar0.set_label("Intensity")

# Plot the post-typhoon image.
img1 = axes[1].imshow(post_typhoon, cmap='gray')
axes[1].set_title("Post-Typhoon Image")
axes[1].axis("off")
cbar1 = fig.colorbar(img1, ax=axes[1], orientation='vertical', fraction=0.046, pad=0.04)
cbar1.set_label("Intensity")

# Plot the difference image
img2 = axes[2].imshow(diff_img_1012_1103, cmap='gray')
axes[2].set_title("Difference (Post - Pre)")
axes[2].axis("off")
cbar2 = fig.colorbar(img2, ax=axes[2], orientation='vertical', fraction=0.046, pad=0.04)
cbar2.set_label("Difference Value")

# Adjust layout and display plots
plt.tight_layout()
plt.show()

The histogram of the difference image shows that the distribution of pixel values is normally distributed. Therefore, it seems possible to consider both sides as significant changes.

In [ ]:
# Plot a histogram of differences
plt.figure(figsize=(10, 6))
plt.hist(diff_img_1012_1103.ravel(), bins=100, color='blue', alpha=0.7)
plt.title("Histogram of Difference between Pre and Post typhoon image")
plt.xlabel("Difference Value")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

### 2.3.2. Detecting Significant Changes in Difference Images

This section presents a procedure for detecting areas of water body change by considering 2.5% ($\mu \pm 2\sigma$) on each side of the pixel value distribution as a significant change.

##### Calculate threshold by mean and standard deviation

Calculate the pixel value mean (`mean_diff`) and standard deviation (`std_diff`) of the difference images.

In the following cells:
- lower_bound and upper_bound with μ±2σ.
- Pixels within `[lower_bound, upper_bound]` are considered “no change” and pixels outside this range [`< lower_bound` or `> upper_bound`] are considered “changed”.
- This method follows the assumption of a normal distribution and assumes that most pixels within ±2σ represent no change.

Note: `diff_img_1012_1103` (the output of the previous section) represents the pixel-by-pixel difference between pre-typhoon and post-typhoon images.

In [ ]:
# Calculate mean and standard deviation
mean_diff = np.mean(diff_img_1012_1103)
std_diff = np.std(diff_img_1012_1103)

# Define threshold with μ±2σ
lower_bound = mean_diff - 2 * std_diff
upper_bound = mean_diff + 2 * std_diff

##### Apply a threshold to classify changed and no change areas.

Create a binary map (`changed_area`) where each pixel is labeled as either "changed" or "no change".

In [ ]:
# Apply a threshold of μ±2σ
changed_area = ((diff_img_1012_1103 < lower_bound) | (diff_img_1012_1103 > upper_bound)).astype(np.uint8)  # 1 for changed, 0 for no-change

##### Visualizing Classification Results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

# `changed_area` is binary change map data, where "changed" is 1 and "no change" is 0.

fig, ax = plt.subplots(figsize=(12, 12))

# Plot the results using a gray color map.
ax.imshow(changed_area, cmap=ListedColormap(['black', 'white']))
ax.set_title("Change Map between Pre-typhoon and Post-typhoon (μ ± 2σ Threshold)")
ax.axis("off")

# Create a custom legend with “no change” and “change” colors.
no_change_patch = mpatches.Patch(color="black", label="No Change")  # Black is no change
changed_patch = mpatches.Patch(color="white", label="Changed")      # White is change
plt.legend(handles=[no_change_patch, changed_patch], loc="lower right", borderaxespad=1)

# Display plot
plt.show()

Changes in the water area due to waterlogging of the dam were detected as “changed”.

Pixels that are considered “changed” are scattered not only in the dam but also in the surrounding area, but these are noise. Since the radar incidence angle (the difference between the slope angle of the terrain and the altitude angle of the satellite) affects the observed values of the synthetic aperture radar, pre-processing to compensate for this effect may be necessary. [ALOS-2/PALSAR-2 data product L2.2 performs slope gradient correction processing, but it is processed only for ScanSAR and is released free of charge](https://www.eorc.jaxa.jp/ALOS/en/dataset/palsar2_l22_e.htm), and standard products are not provided.

It is also important to exclude noise as described above by defining the region of interest in advance. For example, the shape of the water surface in a completely waterlogged condition could be stored in advance as a polygon layer in the GIS, and the change detection analysis could be processed only inside that polygon. It may also be useful to treat neighboring pixels as clusters and consider clusters smaller than a certain size as noise.

### 2.3.3. (Reference) Compare with Sentinel-2 optical satellite imagery.

#### Sample location 1

**Difference between Pre-Typhoon image (observation date: October 11, 2019) and Post-Typhoon image (observation date: November 03, 2019)** 

![Changed-area1](new-img_Diff_Pre-Post.PNG)

| ALOS-2 image (filtered image) | Sentinel-2 image |
|-------------------------------|------------------|
| Before Typhoon (observation date: October 11, 2019) | Before Typhoon (observation date: October 10-11, 2019) |
| ![ALOS-2_pre1](validate-pre1.png) | ![Sentinel-2_pre1](sentinel2-pre1.png) |
| After Typhoon (observation date: November 3, 2019) | After Typhoon (observation date: October 28 - November 03, 2019) |
| ![ALOS-2_post1](validate-post1.png) | ![Sentinel-2_post1](sentinel2-post1.png) |

#### Sample location 2

**Difference between Pre-Typhoon image (observation date: October 11, 2019) and Post-Typhoon image (observation date: November 03, 2019)**

![Changed-area2](new-img_Diff_Pre-Post2.PNG)

| ALOS-2 image (filtered image) | Sentinel-2 image |
|-------------------------------|------------------|
| Before Typhoon (observation date: October 11, 2019) | Before Typhoon (observation date: October 10-11, 2019) |
| ![ALOS-2_pre2](validate-pre2.png) | ![Sentinel-2_pre2](sentinel2-pre2.png) |
| After Typhoon (observation date: November 3, 2019) | After Typhoon (observation date: October 28 - November 03, 2019) |
| ![ALOS-2_post2](validate-post2.png) | ![Sentinel-2_post2](sentinel2-post2.png) |


## 2.4 Advantages and uses of this exercise procedure

### Remote monitoring and surveys

- This is a hint for use in monitoring infrastructure in remote areas such as the Yamba Dam. In recent years, with the development of IoT and satellite communications, simply installing sensors may be sufficient. However, when installing sensors everywhere is costly, combining it with satellite data is effective.

- In areas where infrastructure is being developed for the first time, there is no information necessary to plan the installation of sensors in the first place. In such cases, being able to analyze the dynamics **even roughly** using satellite data will contribute to reducing the investigation costs required for planning.